## Data Exploration - Outlier Detection

#### Objective: Ability to read clean and detect outliers in data

In [1]:
import json
import pandas as pd

In [2]:
df = pd.read_csv('/Users/admin/Documents/VS Code Learning/DataExplorationProjects/DataOutlierDetectionsCSV/raw_data.csv')

In [3]:
df.sample(n=10, random_state=20)

,member_unique_id,gender,dob,eligible_year,eligible_month,affiliation_type,pbp_group,plan_name,npi,line_of_business,...,pcp_ffs,plan_premium,prof,reinsurance,risk_score_partd,rx,rx_rebates,rx_with_rebates,rx_without_rebates,spec_cap
7458,7459,F,08/12/1933,2020,202006,VIPcare,NON-SNP,NaN,63,HMO,...,$0.00,"$20,200.00",$0.00,$8.06,$0.00,$12.06,$0.00,$13.21,$13.14,$786.16
8047,8048,F,06/05/1961,2020,202006,Affiliate,NON-SNP,MEDICARE - CAREFREE,69,HMO,...,$0.00,"$15,326.47",$0.00,$9.39,$0.00,"$69,247.49",$0.00,"$68,915.21","$71,028.34",$909.74
8052,8053,M,25/12/1949,2020,202006,Affiliate,NON-SNP,MEDICARE - CAREFREE,69,HMO,...,$0.00,"$16,007.97",$0.00,$8.21,$0.00,$0.00,$0.00,$0.00,$0.00,$897.84
2522,2523,F,10/02/1953,2020,202006,Affiliate,NON-SNP,MEDICARE - CAREFREE,20,HMO,...,$0.00,"$4,177.98",$730.58,$9.20,$0.00,$289.80,$0.00,$291.99,$293.65,$866.42
4113,4114,M,09/11/1948,2020,202006,Affiliate,NON-SNP,NaN,31,HMO,...,$0.00,"$10,032.17",$0.00,$8.51,$0.00,"$15,681.50",$0.00,"$13,977.64","$14,058.75",$980.47
7347,7348,F,09/04/1952,2020,202006,Affiliate,NON-SNP,MEDICARE - CAREFREE,62,HMO,...,$0.00,"$3,067.57",$0.00,$9.63,$0.00,$0.00,$0.00,$0.00,$0.00,$629.54
569,570,M,21/12/1946,2020,202006,Affiliate,NON-SNP,NaN,5,HMO,...,$0.00,"$8,089.97",$0.00,$9.38,$0.00,$454.15,$0.00,$436.94,$453.67,$770.56
6491,6492,M,20/04/1941,2020,202006,VIPcare,NON-SNP,NaN,57,HMO,...,$0.00,"$7,075.97",$0.00,$9.41,$0.00,$95.87,$0.00,$95.12,$95.02,$675.93
8630,8631,F,26/09/1946,2020,202006,VIPcare,NON-SNP,MEDICARE - CAREFREE,75,HMO,...,$0.00,"$2,584.05",$0.00,$9.24,$0.00,$458.20,$0.00,$474.33,$485.74,$659.55
9942,9943,F,09/04/1949,2020,202006,VIPcare,D-SNP,MEDICARE - CARENEEDS,87,HMO,...,$0.00,"$7,819.41",$0.00,$9.33,$0.00,$233.86,$0.00,$210.18,$239.30,$692.63


In [4]:
df.shape

(10530, 28)

In [9]:
# transform to Brazilian timezone
df["EnqueuedTimeUtc"] = pd.to_datetime(arg=df["EnqueuedTimeUtc"]).dt.tz_convert(tz="Brazil/East")

In [10]:
df["EventName"].value_counts()

EventName
DynamicPrice_Result    32
CurateOffer_Result      5
Name: count, dtype: int64

In [11]:
#DynamicPriceOption

dynamic_price_events = df[df["EventName"] == "DynamicPrice_Result"]
dynamic_price_events.shape

(32, 3)

In [12]:
dynamic_price_events.head()

,EnqueuedTimeUtc,EventName,Payload
0,2021-09-05 05:04:08-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
1,2021-08-18 08:43:23-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."
2,2021-09-05 06:04:04-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
4,2021-09-05 05:03:28-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
5,2021-08-18 08:44:00-03:00,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."


In [15]:
def dynamic_price_option(row):
    """Transform the JSON payload into a dynamic price option dataframe."""
    enqueued_time = row["EnqueuedTimeUtc"].to_pydatetime().strftime("%d/%m/%y")
    payload_row = row["Payload"]
    payload_row = json.loads(payload_row)
    result = []
    if payload_row["provider"] == "ApplyDynamicPricePerOption":
        provider = f"\"{payload_row['provider']}\""
        offer_id = f"\"{payload_row['offerId']}\""
        for unique_option in payload_row["algorithmOutput"]:
            price_option = {
                "Provider": provider,
                "OfferId": offer_id,
                "UniqueOptionId": f"\"{unique_option['uniqueOptionId']}\"",
                "BestPrice": unique_option["bestPrice"],
                "EnqueuedTimeSP": enqueued_time
            }
            result.append(price_option)
    return result


lst = dynamic_price_events.apply(dynamic_price_option, axis=1)
flatten = [x for xs in lst for x in xs]
dynamic_price_option_payload = pd.DataFrame(flatten)

In [16]:
dynamic_price_option_payload.head()

,Provider,OfferId,UniqueOptionId,BestPrice,EnqueuedTimeSP
0,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""b0e296a9-0590-f0e0-8211-243a2ededb12""",92.45,18/08/21
1,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""d6562c24-0b37-5fb4-8275-65b7b8b47b87""",92.45,18/08/21
2,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""8d0f9262-f543-d0c8-a869-33985ae3ecda""",92.45,18/08/21
3,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""151e59ac-761a-96f5-d2b9-882037a9fd28""",94.60,18/08/21
4,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""3cd346f4-d297-7568-2e50-d43a8e2fd0a9""",94.60,18/08/21


In [17]:
#DynamicPriceRange
def dynamic_price_range(row):
    """Transform the JSON payload into a dynamic price range dataframe."""
    enqueued_time = row["EnqueuedTimeUtc"].to_pydatetime().strftime("%d/%m/%y")
    payload_row = row["Payload"]
    payload_row = json.loads(payload_row)
    result = []
    if payload_row["provider"] == "ApplyDynamicPriceRange":
        provider = f"\"{payload_row['provider']}\""
        offer_id = f"\"{payload_row['offerId']}\""
        algorithm_output = payload_row["algorithmOutput"]
        price_range = {
            "Provider": provider,
            "OfferId": offer_id,
            "MinGlobal": algorithm_output["min_global"],
            "MinRecommended": algorithm_output["min_recommended"],
            "MaxRecommended": algorithm_output["max_recommended"],
            "DifferenceMinRecommendMinTheory": algorithm_output["differenceMinRecommendMinTheory"],
            "EnqueuedTimeSP": enqueued_time
        }
        result.append(price_range)
    return result


lst = dynamic_price_events.apply(dynamic_price_range, axis=1)
flatten = [x for xs in lst for x in xs]
dynamic_price_range_payload = pd.DataFrame(flatten)

In [18]:
dynamic_price_range_payload.head()

,Provider,OfferId,MinGlobal,MinRecommended,MaxRecommended,DifferenceMinRecommendMinTheory,EnqueuedTimeSP
0,"""ApplyDynamicPriceRange""","""a6611d55-9624-4381-8cdd-323ee3689241""",85.00,87.20,97.65,2.2,05/09/21
1,"""ApplyDynamicPriceRange""","""b8c636fa-8241-47dc-ac40-bdf438a04d9c""",85.00,87.20,97.65,2.2,05/09/21
2,"""ApplyDynamicPriceRange""","""3d32f7fb-396d-4d3f-b673-dea1f7dc41b7""",85.00,87.20,97.65,2.2,05/09/21
3,"""ApplyDynamicPriceRange""","""329194f3-95a4-45ef-b3d0-2796f74ce2a0""",85.00,87.20,97.65,2.2,05/09/21
4,"""ApplyDynamicPriceRange""","""fdcfde5c-113d-4a59-9ae0-8bc31e2943d8""",87.35,89.25,99.95,1.9,05/09/21


In [20]:
#CurratedOfferOptions
curated_offers = df[df["EventName"] == "CurateOffer_Result"]
curated_offers.shape


(5, 3)

In [21]:
curated_offers.head()

,EnqueuedTimeUtc,EventName,Payload
3,2021-08-25 02:02:55-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""149f..."
13,2021-08-25 06:03:29-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""c99a..."
15,2021-08-25 02:02:18-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""d668..."
17,2021-08-25 06:02:29-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""0a06..."
35,2021-08-25 06:03:14-03:00,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""135d..."


In [22]:
def curated_offer(row):
    """Transform JSON payload into curated offer dataframe."""
    enqueued_time = row["EnqueuedTimeUtc"].to_pydatetime().strftime("%d/%m/%y")
    payload_row = row["Payload"]
    payload_row = json.loads(payload_row)
    result = []
    for el in payload_row:
        curation_provider = f"\"{el['curationProvider']}\""
        offer_id = f"\"{el['offerId']}\""
        dealer_id = f"\"{el['dealerId']}\""
        for option in el["options"]:
            offer_options = {
                "CurationProvider": curation_provider,
                "OfferId": offer_id,
                "DealerId": dealer_id,
                "UniqueOptionId": f"\"{option['uniqueOptionId']}\"",
                "OptionId": f"\"{option['optionId']}\"",
                "IsMobileDealer": option["isMobileDealer"],
                "IsOpen": option["isOpen"],
                "Eta": f"\"{option['eta']}\"",
                "ChamaScore": option["chamaScore"],
                "ProductBrand": f"\"{option['productBrand']}\"",
                "IsWinner": option["isWinner"],
                "MinimumPrice": option["minimumPrice"],
                "MaximumPrice": option["maximumPrice"],
                "DynamicPrice": option["dynamicPrice"],
                "FinalPrice": option["finalPrice"]
            }
            if "defeatPrimaryReason" in option:
                offer_options["DefeatPrimaryReason"] = f"\"{option['defeatPrimaryReason']}\""
                offer_options["DefeatReasons"] = f"\"{option['defeatReasons']}\""
            else:
                offer_options["DefeatPrimaryReason"] = "\"\""
                offer_options["DefeatReasons"] = "\"\""
            offer_options["EnqueuedTimeSP"] = enqueued_time
            result.append(offer_options)
    return result


flatten = [x for xs in curated_offers.apply(curated_offer, axis=1) for x in xs]
curated_offers_payload = pd.DataFrame(flatten)

In [23]:
curated_offers_payload.head()

,CurationProvider,OfferId,DealerId,UniqueOptionId,OptionId,IsMobileDealer,IsOpen,Eta,ChamaScore,ProductBrand,IsWinner,MinimumPrice,MaximumPrice,DynamicPrice,FinalPrice,DefeatPrimaryReason,DefeatReasons,EnqueuedTimeSP
0,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""6517""","""b0e296a9-0590-f0e0-8211-243a2ededb12""","""6517 || dd839e4c-9f84-45eb-9cb2-9069fecf70f2""",True,True,"""1:00""",8.0,"""ULTRAGAZ""",True,90.00,180.00,91.90,91.90,"""""","""""",25/08/21
1,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""6517""","""d6562c24-0b37-5fb4-8275-65b7b8b47b87""","""6517 || 6517""",False,False,"""0:01""",8.0,"""ULTRAGAZ""",False,90.00,180.00,91.90,91.90,"""Closed""","""['Closed', 'HasDriverInOffer']""",25/08/21
2,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""9047""","""8d0f9262-f543-d0c8-a869-33985ae3ecda""","""9047 || 9047 || ULTRAGAZ""",False,False,"""1:00""",9.0,"""ULTRAGAZ""",False,99.00,198.00,99.95,99.95,"""Closed""","""['Closed', 'HigherPrice', 'HasDriverInOffer']""",25/08/21
3,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""9047""","""3cd346f4-d297-7568-2e50-d43a8e2fd0a9""","""9047 || 9047 || CONSIGAZ""",False,False,"""1:00""",9.0,"""CONSIGAZ""",False,89.99,179.98,91.89,91.89,"""Closed""","""['Closed', 'HigherPrice', 'HigherETA']""",25/08/21
4,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""9047""","""577e4bbd-f49d-ac23-56a6-e70072a05229""","""9047 || 9047 || LIQUIGAS""",False,False,"""1:00""",9.0,"""LIQUIGAS""",False,92.00,184.00,93.90,93.90,"""Closed""","""['Closed', 'HigherPrice', 'HigherETA']""",25/08/21


In [ ]:
# dynamic_price_option_payload.to_csv("dynamic_price_option_payload.csv", index=False)
# dynamic_price_range_payload.to_csv("dynamic_price_range_payload.csv", index=False)
# curated_offers_payload.to_csv("curated_offers_payload.csv", index=False)